In [1]:
import pandas as pd
import requests
import pdfplumber
import re
import numpy as np
import gspread
import csv
import time

/opt/homebrew/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [11]:
credentials = {
  "type": "",
  "project_id": "",
  "private_key_id": "",
  "private_key": "",
  "client_email": "",
  "client_id": "",
  "auth_uri": "",
  "token_uri": "",
  "auth_provider_x509_cert_url": "",
  "client_x509_cert_url": ""
}

sa = gspread.service_account_from_dict(credentials)
sh = sa.open('Personal Fin')

combine = pd.DataFrame()

def extract_dbs(filename1):
    # must use sep='\t' when importing csv else you will get error tokenising the data
    dbs_statement = pd.read_csv(filename1, skiprows=17,sep='\t')
    df = dbs_statement['Transaction Date,Reference,Debit Amount,Credit Amount,Transaction Ref1,Transaction Ref2,Transaction Ref3'].str.split(',',expand=True)

    df['Description'] = df[4] + df[5] + df[6]
    df.drop([1,4,5,6,7],inplace=True, axis=1)
    df.rename({
        0:'Date',
        2:'Debit',
        3:'Credit'
    },inplace=True,axis=1)
    
    df['part1'] = df['Date'].str.split(' ').str.get(0)
    df['part2'] = df['Date'].str.split(' ').str.get(1)
    df['Date'] = df['part1']+' '+df['part2']
    
    #any row on debit side should be negative
    df['Debit'] = '('+df['Debit']+')'

    #fill empty debit cells with credit values
    df['Debit'] = np.where(df['Debit'] == '( )', df['Credit'], df['Debit'])
    df.drop('Credit',inplace=True,axis=1)
    df.rename({'Debit':'Amt'},inplace=True, axis=1)

    df.drop(['part1','part2'],inplace=True,axis=1)
    df['Bank'] = 'DBS'
    
    
    return df

def extract_citi(filename2):
    global table
    
    trans = pd.DataFrame()

    with pdfplumber.open(filename2) as pdf:
        pages = pdf.pages

        for i,pg in enumerate(pages):    
            text = pages[i].extract_text()
            # print(f'{i} --- {text}')

            new_re = re.compile(r'(?<!\d)(\d{2})[A-Z]{3} [A-Z]')

            for line in text.split('\n'):
                if new_re.match(line):
                    line_item = pd.Series(line, dtype=pd.StringDtype())
                    trans = pd.concat([trans,line_item])

    trans = pd.DataFrame(trans)
    trans.reset_index(inplace=True)
    trans.drop('index',inplace=True,axis=1)

    date = trans[0].apply(lambda x:x.split()[0])

    ### take out of a list
    description = trans[0].apply(lambda x:x.split()[1:len(x)])

    output_des= list()
    i=0

    for x in description:
        split = ' '.join(map(str, x))
        output_des.insert(i,split)
        i=i+1

    description = pd.DataFrame(output_des)

    amount = trans[0].apply(lambda x:x.split()[-1])
    
    table = pd.concat([date,description,amount],axis=1)
    table.columns=["1","2","3"]
    table.rename(columns = {'1':'Date','2':'Description','3':'Amt',}, inplace = True)
    

    table['Bank'] = 'Citi'
    
    # remove the brackets in values replace with negative sign 
    # so that i can convert positive to negative and negative to positive

    table['Amt'].astype(str)

    for index, amt in enumerate(table['Amt']):
        fullstring = "{x}".format(x=table['Amt'])
        substring = "("

        if substring in fullstring:
            table['Amt'][index] = table['Amt'][index].replace("(","-")
            table['Amt'][index] = table['Amt'][index].replace(")","")

    # convert to float and also, any phrases that are present are turned into nan
    table['Amt']
    table['Amt'] = pd.to_numeric(table['Amt'], errors='coerce')

    table['Amt'].astype(float)
    table['Amt']= -table['Amt']
    
    #have to convert nan to empty '' else will throw error
    table['Amt'] = table['Amt'].fillna('')

    return table

def combine_statements(filename1,filename2):
    
    global combine
    
    dbs = extract_dbs(filename1)
    citi = extract_citi(filename2)
    combine = pd.concat([dbs,citi],axis=0)
   
    
    category = [
    #transport
    combine.Description.str.lower().str.contains('parking|bus'), 
    #eating out
    combine.Description.str.lower().str.contains("|naturesnutrition|collin's|guzmanygomez"), 
    #phone bill
    combine.Description.str.lower().str.contains('giga'), 
    #groceries
    combine.Description.str.lower().str.contains('fairpricefinest|esso-cheersbyfp|marks&spencer|ntuc|giant|hypervivo|coldstorage'),
    #health and wellness
    combine.Description.str.lower().str.contains('cut&curl|qbhouse|guardian|watson'), 
    #entertainment
    combine.Description.str.lower().str.contains('GV'), 
    #shopping
    combine.Description.str.lower().str.contains('muji|uniqlo|decathlon|lazada|charles&keithps|asos|shopee'), 
    #fee
    combine.Description.str.lower().str.contains('interest'), 
    #misc
    combine.Description.str.lower().str.contains('fiverr'),
    #transfers
    combine.Description.str.lower().str.contains('top-up to paylah!|maxed out from paylah!|i-bank')
    ]

    category_values = [
        'Transport',
        'Eating out',
        'Phone bill',
        'Groceries',
        'Health & wellness',
        'Entertainment',
        'Shopping',
        'Fees',
        'Misc',
        'Transfers'
    ]
    
    
    priority = [
    #Shouln't have
    combine.Description.str.lower().str.contains("deliveroosi"), 
    #Nice to have
    combine.Description.str.lower().str.contains("lazada"), 
    #Essential
    combine.Description.str.lower().str.contains("bus/mrt|ccyconversion")
    ]

    priority_values = [
        "Shouldn't have",
        "Nice to have",
        "Essential"
    ]

    ### add additional columns here ###
    
    combine['Category'] = np.select(category, category_values, default='?')
    combine['Priority'] = np.select(priority, priority_values, default='?')
    combine['Include Txn?'] = 'Yes'
    
    new_index = ['Date', 'Description', 'Amt', 'Bank', 'Category','Priority','Include Txn?']
    combine = combine.reindex(new_index, axis=1)
    
    
    return combine

In [12]:
def export_to_sheet(month, combine):
    
    #select worksheet
    wks = sh.worksheet(f'{month}')
    
    #export data from pandas to google sheets
    # you need raw=False else you will see ' infront of your values and you cant do sum or anything
    
    wks.update('A6',[combine.columns.values.tolist()] + combine.values.tolist(),raw=False)
    
    #format cells
    wks.format('1:100', {'textFormat': {'bold': False}})
    wks.format('6', {'textFormat': {'bold': True}})
    wks.format("A1", {
        "backgroundColor": {
          "red": 0.0,
          "green": 0.0,
          "blue": 0.0
        },
        "horizontalAlignment": "CENTER",
        "textFormat": {
          "foregroundColor": {
            "red": 1.0,
            "green": 1.0,
            "blue": 1.0
          },
          "fontSize": 12,
          "bold": True
        }
    })
    wks.format("A6:G6", {
        "backgroundColor": {
          "red": 0.0,
          "green": 0.0,
          "blue": 0.0
        },
        "horizontalAlignment": "CENTER",
        "textFormat": {
          "foregroundColor": {
            "red": 1.0,
            "green": 1.0,
            "blue": 1.0
          },
          "fontSize": 12,
          "bold": True
        }
    })
    
    print('data exported to sheets')
    


# Execution

In [13]:
combine_statements('DBS_oct22.csv','citi_oct2022.pdf')
export_to_sheet('september',combine)

data exported to sheets
